In [ ]:
#создаем новый признак 'Город'
#через лямбду вырезаем первое слово и сохраняем в новый признак 
hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(lambda x: ' '.join(x.split(' ')[:1]))
#прописываем функцию которая преобразует новый признак "Город" в 4 категории 
def get_category(arg):
    million_cities = [
        'Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск', 'Омск', \
            'Самара', 'Ростов-на-Дону','Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград' 
        ]
    if arg in million_cities:
        return 'город-миллионник'
    elif arg == 'Москва':
        return 'Москва'
    elif arg == 'Санкт-Петербург':
        return 'Санкт-Петербург'
    else:
        return "другие"  
#применяем данную функцию     
hh_data['Город'] = hh_data['Город'].apply(get_category)
#проверяем, должно быть 4 уникальных значений
print(hh_data['Город'].nunique()) #4

#создаем новый признак 'Готовность к переезду'
hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(lambda x: x.split(',')[1])
#прописываем функцию которая преобразует новый признак "Готовность к переезду"
def get_ready_to_moov(arg):
    if ('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    elif 'хочу' in arg:
        return True
    else:
        return True
#применяем данную функцию
hh_data['Готовность к переезду'] = hh_data['Готовность к переезду'].apply(get_ready_to_moov)
print(hh_data['Готовность к переезду'].nunique())

#создаем новый признак 'Готовность к командировкам'
hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(lambda x: ' '.join(x.split(',')[2:]))
def get_ready_for_trips(arg):
    if ('командировка' in arg):
        if ('не готов к командировкам' in arg) or ('не готова к командировкам' in arg):
            return False
        else:

            return True
    else:
        return False
#применяем данную функцию
hh_data['Готовность к командировкам'] = hh_data['Готовность к командировкам'].apply(get_ready_for_trips)
print(hh_data['Готовность к командировкам'].nunique())


In [ ]:
def get_city(arg):
    million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород',
                      'Казань', 'Челябинск', 'Омск', 'Самара', 'Ростов-на-Дону',
                      'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград'
                      ]
    city = arg.split(' , ')[0]
    if (city == 'Москва') or (city == 'Санкт-Петербург'):
        return city
    elif city in million_cities:
        return 'город миллионник'
    else:
        return 'другие'


def get_ready_to_move(arg):
    if ('не готов к переезду' in arg) or ('не готова к переезду' in arg):
        return False
    elif 'хочу' in arg:
        return True
    else:
        return True


def get_ready_for_bisiness_trips(arg):
    if ('командировка' in arg):
        if ('не готов к командировкам' in arg) or ('не готова к командировкам' in arg):
            return False
        else:

            return True
    else:
        return False


hh_data['Город'] = hh_data['Город, переезд, командировки'].apply(get_city)
hh_data['Готовность к переезду'] = hh_data['Город, переезд, командировки'].apply(get_ready_to_move)
hh_data['Готовность к командировкам'] = hh_data['Город, переезд, командировки'].apply(get_ready_for_bisiness_trips)
hh_data = hh_data.drop('Город, переезд, командировки', axis=1)
print(round(hh_data['Город'].value_counts(normalize=True)['Санкт-Петербург'] * 100))
print(round(hh_data[
                hh_data['Готовность к переезду'] & hh_data['Готовность к командировкам']
                ].shape[0] / hh_data.shape[0] * 100))

In [ ]:
rates = pd.read_csv('data/ExchangeRates.csv')
rates = rates.drop(['per','time', 'vol'], axis=1)

rates['date'] = pd.to_datetime(rates['date'], dayfirst=True).dt.date
hh_data['Обновление резюме'] = pd.to_datetime(hh_data['Обновление резюме'], dayfirst=True).dt.date

# создаём столбцы с названием валюты и желаемой зп
hh_data['валюта'] = hh_data['ЗП'].apply(lambda x: str(x.split(' ')[1]))
hh_data['желаемая ЗП'] = hh_data['ЗП'].apply(lambda y: y.split(' ')[0])
#убираем точки в названии валюты
hh_data['желаемая ЗП'].replace('.', '')
#Создаем словарь для валюты в ISO
currency_dict = {
        'USD': 'USD', 'KZT': 'KZT',
        'грн': 'UAH', 'белруб': 'BYN',
        'EUR': 'EUR', 'KGS': 'KGS',
        'сум': 'UZS', 'AZN': 'AZN', 'руб': 'RUB'
    }
#Применяем словарь к таблице
hh_data['валюта'] = hh_data['валюта'].map(currency_dict)
#hh_data.dropna(inplace=True)
# соединим таблицы hh_data и rates
hh_data = hh_data.merge(right = rates,
                         how = 'left',
                         left_on=['валюта', 'Обновление резюме'],
                         right_on=['currency', 'date',])
#заполняем появившиеся пропуски в столбцах close и proportion единицей (1) 1 с помощью метода fillna().
hh_data = hh_data.fillna(value={'close': 1, 'proportion': 1})
#преобразуем тип данных в столбце желаемая ЗП'
hh_data['желаемая ЗП'] = hh_data['желаемая ЗП'].astype('float64')
#создаем столбец 'ЗП (руб)'
hh_data['ЗП (руб)'] = hh_data['желаемая ЗП']*hh_data['close']/hh_data['proportion']
#удаляем исходный столбец заработной платы «ЗП» и промежуточные столбцы
hh_data = hh_data.drop(['ЗП','currency', 'date', 'валюта'], axis=1)

In [ ]:
def get_experience(arg):
    if arg is np.nan or arg == 'Не указано':
        return None
    year_words=['год', 'года', 'лет']
    month_words=['месяц', 'месяца', 'месяцев']
    arg_splitted = arg.split(' ')[:7]
    years = 0
    months = 0
    for index, item in enumerate (arg_splitted):
        if item in year_words:
            years = int(arg_splitted[index-1])
        if item in month_words:
            months = int(arg_splitted[index-1])
    return int(years*12 + months)


hh_df['Опыт работы (месяц)'] = hh_data['Опыт работы'].apply(get_experience)
hh_df = hh_df.drop('Опыт работы', axis=1)

In [ ]:
#создаём словарь со столбцами, в которых надо удалить строки с пропусками 
values = {
    'Последняя/нынешняя должность':hh_df['Последняя/нынешняя должность'].median(),
    'BloodPressure':combine_data['BloodPressure'].median(),
    'Glucose':combine_data['Glucose'].median(),
    'BMI':combine_data['BMI'].median()
}
#заполняем оставшиеся записи константами в соответствии со словарем values
hh_df = hh_df.fillna(values)
#выводим результирующую долю пропусков
round(diabetes['SkinThickness'].mean(), 1)

In [ ]:

hh_df= hh_df['Последняя/нынешняя должность', 'Последнее/нынешнее место работы'].dropna(how='any', thresh=thresh, axis=0)
print(hh_df.shape)